In [20]:
from math import *
import pandas as pd
import numpy as np

In [21]:
data = pd.read_csv('ANUBHAV JHA - heart_disease_uci.csv')

In [22]:
data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [23]:
data.shape,
data.isnull().sum()

,0
id,0
age,0
sex,0
dataset,0
cp,0
trestbps,59
chol,30
fbs,90
restecg,2
thalch,55


In [24]:

for col in data.columns:

    if data[col].dtype in ['float64', 'int64']:
      data[col] = data[col].fillna(data[col].mean())
    else:
      data[col] = data[col].fillna(data[col].mode()[0])



<ipython-input-24-ba54ab1cb57b>:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].fillna(data[col].mode()[0])


In [25]:
for col in data.columns:
  unique_count = data[col].nunique()
  print(f"Column '{col}' has {unique_count} unique entries.")

Column 'id' has 920 unique entries.
Column 'age' has 50 unique entries.
Column 'sex' has 2 unique entries.
Column 'dataset' has 4 unique entries.
Column 'cp' has 4 unique entries.
Column 'trestbps' has 62 unique entries.
Column 'chol' has 218 unique entries.
Column 'fbs' has 2 unique entries.
Column 'restecg' has 3 unique entries.
Column 'thalch' has 120 unique entries.
Column 'exang' has 2 unique entries.
Column 'oldpeak' has 54 unique entries.
Column 'slope' has 3 unique entries.
Column 'ca' has 5 unique entries.
Column 'thal' has 3 unique entries.
Column 'num' has 5 unique entries.


In [26]:
# from above we can classify it as
binary_cols = ['sex','fbs','exang']
ordinal_cols = ['cp', 'restecg', 'slope', 'thal','ca']
numerical_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']


In [27]:
data.drop_duplicates(inplace=True)
data.shape

(920, 16)

In [28]:
data.drop('id',axis = 1, inplace = True)
data.head()

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [29]:

def numeric_distance(data, cols, h):
    num_rows = len(data)
    distance = np.zeros((num_rows, num_rows))

    for i in range(num_rows):
        for j in range(i + 1, num_rows):
            # Calculate distance between row i and row j
            row1 = data.loc[i, cols].values  # Get values for specified columns
            row2 = data.loc[j, cols].values
            if h == 1:
                dist = np.sum(np.abs(row1 - row2))  # Manhattan distance
            elif h == 2:
                dist = np.sqrt(np.sum((row1 - row2)**2))  # Euclidean distance
            else:
                dist = np.sum(np.abs(row1 - row2)**h)**(1/h)  # Minkowski distance

            distance[i][j] = dist
            distance[j][i] = dist  # Ensure symmetry

    return distance

In [30]:
def cosine_similarity(data):

    num_cols = len(numerical_cols)
    distance = np.zeros((num_cols, num_cols))

    for i in range(num_cols):
        for j in range(i + 1, num_cols):
            col1 = numerical_cols[i]
            col2 = numerical_cols[j]

            vec1 = data[col1]
            vec2 = data[col2]

            # Compute dot product
            dot_product = np.dot(vec1, vec2)


            norm_vec1 = np.linalg.norm(vec1)
            norm_vec2 = np.linalg.norm(vec2)

            # No division by 0
            if norm_vec1 == 0 or norm_vec2 == 0:
                distance[i][j] = 0
            else:
                distance[i][j] = dot_product / (norm_vec1 * norm_vec2)

            distance[j][i] = distance[i][j]

    return distance

cosine_similarity(data)

array([[0.        , 0.98104635, 0.85705868, 0.95781551, 0.66398806],
       [0.98104635, 0.        , 0.87481072, 0.97168975, 0.65152422],
       [0.85705868, 0.87481072, 0.        , 0.88260516, 0.57886518],
       [0.95781551, 0.97168975, 0.88260516, 0.        , 0.60959352],
       [0.66398806, 0.65152422, 0.57886518, 0.60959352, 0.        ]])

In [31]:
def nominal_distance(data, cols):
    num_cols = len(cols)
    similarity_matrix = np.zeros((num_cols, num_cols))

    for i in range(num_cols):
        for j in range(i + 1, num_cols):

            col1 = cols[i]
            col2 = cols[j]

            if col1 == col2:
              similarity_matrix[i][j] = 1
            else:
              similarity_matrix[i][j] = 0

            similarity_matrix[j][i] = similarity_matrix[i][j]

    return similarity_matrix

In [ ]:
import matplotlib.pyplot as plt

b=numeric_distance(data,numerical_cols,1)
c=numeric_distance(data,numerical_cols,2)
d=numeric_distance(data,numerical_cols,3)

print(f"Euclidean Distance : \n{b}")

print("Heatmap :")

plt.imshow(b, cmap='hot', interpolation='nearest')
plt.show()

print("\n")
print(f"Manhattan Distance : \n{c}")

print("Heatmap :")

plt.imshow(c, cmap='hot', interpolation='nearest')
plt.show()


print("Minkowski Distance : \n{d}")
print(d)
plt.imshow(c, cmap='hot', interpolation='nearest')
plt.show()


In [ ]:
from sklearn.preprocessing import LabelEncoder
LabelEncode = LabelEncoder()

for i in ordinal_cols:
  df[f"{i}_encoded"] = LabelEncode.fit_transform(df[i])

# # from above we can classify it as
# binary_cols = ['sex','fbs','exang']
# ordinal_cols = ['cp', 'restecg', 'slope', 'thal','ca']
# numerical_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']

